# Reading 3ddose file from EGSnrc_TDuong_05.2024

In [11]:
# Giải Thích Quy Trình
# Bước 1: Đọc Dữ Liệu từ File 3ddose:
# Hàm read_3ddose đọc file 3ddose và trả về tọa độ x, y, z, liều và sai số.


# Bước 2: Tính Kích Thước Voxel:
# Hàm calculate_voxel_size nhận vào các tọa độ ranh giới và tính toán kích thước của mỗi voxel.


# Bước 3: Xác định mật độ:
# Ở đây, mật độ của chất liệu được xác định, trong ví dụ này là 1.0 g/cm³.


# Bước 4: Tính Toán Năng Lượng Lắng Đọng:
# Sử dụng kích thước voxel và mật độ để chuyển đổi liều thành năng lượng lắng đọng trong hàm calculate_energy_deposited.

# Bước 5: Vẽ Profile:
# Các hàm vẽ profile giúp bạn hình dung sự phân bố năng lượng lắng đọng dọc theo các trục x, y, và z tại vị trí trung tâm của các trục còn lại.


import numpy as np
from matplotlib import pyplot as plt

# Bước 1: Đọc Dữ Liệu từ File 3ddose
def read_3ddose(file: str):
    counter = 0
    nextline = True
    with open(file) as f:
        while nextline: 
            line = f.readline()
            if counter == 0:  # Đọc kích thước của lưới voxel, ví dụ: 512x512x47
                nxnynz = line.split()
                nx = int(nxnynz[0])
                ny = int(nxnynz[1])
                nz = int(nxnynz[2])
            elif counter == 1:  # Đọc tọa độ ranh giới trên trục x
                x_coords = np.array(line.split(), dtype=float) 
            elif counter == 2:  # Đọc tọa độ ranh giới trên trục y
                y_coords = np.array(line.split(), dtype=float)
            elif counter == 3:  # Đọc tọa độ ranh giới trên trục z
                z_coords = np.array(line.split(), dtype=float)
            elif counter == 4:  # Đọc dữ liệu liều
                doses = np.array(line.split(), dtype=np.float32).reshape((nz, ny, nx))
                doses = np.flip(doses, 1)  # Lật dữ liệu nếu cần
            elif counter == 5:  # Đọc dữ liệu sai số
                uncertainties = np.array(line.split(), dtype=np.float32).reshape((nz, ny, nx))
                uncertainties = np.flip(uncertainties, 1)  # Lật dữ liệu nếu cần
                break
            counter += 1
    print('done')
    return x_coords, y_coords, z_coords, doses, uncertainties

# Bước 2: Tính Kích Thước Voxel
def calculate_voxel_size(x_coords, y_coords, z_coords):
    # Tính kích thước voxel dọc theo mỗi trục
    dx = x_coords[1] - x_coords[0]  # Khoảng cách giữa hai tọa độ liên tiếp trên trục x
    dy = y_coords[1] - y_coords[0]  # Khoảng cách giữa hai tọa độ liên tiếp trên trục y
    dz = z_coords[1] - z_coords[0]  # Khoảng cách giữa hai tọa độ liên tiếp trên trục z
    return (dx, dy, dz)

# Bước 3: Tính Năng Lượng Lắng Đọng
def calculate_energy_deposited(dose, density, voxel_size):
    # Tính thể tích của mỗi voxel trực tiếp từ dx, dy, dz
    voxel_volume = 0.001 * voxel_size[0] * voxel_size[1] * voxel_size[2]  # Chuyển đổi từ cm³ sang m³ bằng cách nhân với 0.001
    
    # Tính khối lượng của mỗi voxel
    voxel_mass = density * voxel_volume  # rho * V
    
    # Chuyển đổi liều thành năng lượng lắng đọng
    energy_deposited = dose * voxel_mass  # D * m
    
    # Tính tổng năng lượng lắng đọng tại tất cả các voxel
    total_energy_deposited = np.sum(energy_deposited)
    
    return energy_deposited, total_energy_deposited

# Bước 1: Đọc dữ liệu từ file
x_coords, y_coords, z_coords, dose, uncertainties = read_3ddose(file_path)

# Bước 2: Tính kích thước voxel
voxel_size = calculate_voxel_size(x_coords, y_coords, z_coords)
# print(f'Voxel size: {voxel_size}')

# Bước 3: Xác định mật độ
density = 1000  # mật độ của chất liệu (kg/m³) - giả sử là nước, nếu không thì cần cập nhật giá trị đúng

# Bước 4: Tính toán năng lượng lắng đọng
energy_deposited_J, total_energy_deposited_J = calculate_energy_deposited(dose, density, voxel_size)

# Chuyển đổi năng lượng từ J sang keV
energy_deposited_keV = energy_deposited_J * 6.242e12
total_energy_deposited_keV = total_energy_deposited_J * 6.242e12

# In kết quả
print(f'Total energy deposited: {total_energy_deposited_J} J')
print(f'Total energy deposited: {total_energy_deposited_keV} keV')

# Bước 5: Vẽ Profile
# Vẽ profile theo trục x tại vị trí trung tâm của các trục y và z
def plot_profile_x(energy, y_index, z_index, unit='J'):
    profile = energy[z_index, y_index, :]
    plt.plot(profile)
    plt.xlabel('Voxel index along X-axis')
    plt.ylabel(f'Energy Deposited ({unit})')
    plt.title(f'Energy Deposited Profile along X-axis in {unit}')
    plt.grid(True)
    plt.show()

# Vẽ profile theo trục y tại vị trí trung tâm của các trục x và z
def plot_profile_y(energy, x_index, z_index, unit='J'):
    profile = energy[z_index, :, x_index]
    plt.plot(profile)
    plt.xlabel('Voxel index along Y-axis')
    plt.ylabel(f'Energy Deposited ({unit})')
    plt.title(f'Energy Deposited Profile along Y-axis in {unit}')
    plt.grid(True)
    plt.show()

# Vẽ profile theo trục z tại vị trí trung tâm của các trục x và y
def plot_profile_z(energy, x_index, y_index, unit='J'):
    profile = energy[:, y_index, x_index]
    plt.plot(profile)
    plt.xlabel('Voxel index along Z-axis')
    plt.ylabel(f'Energy Deposited ({unit})')
    plt.title(f'Energy Deposited Profile along Z-axis in {unit}')
    plt.grid(True)
    plt.show()

# Vị trí để vẽ profile
center_y = y_coords.size // 2
center_z = z_coords.size // 2
center_x = x_coords.size // 2

# Đường dẫn đến file
file_path = 'DPK_Lu177.3ddose'
# file_path = 'DPK_Cu67.3ddose'
# file_path = 'DPK_Sc47.3ddose'
# file_path = 'DPK_Tb161.3ddose'


# Vẽ profile theo năng lượng lắng đọng tính bằng keV
# plot_profile_x(energy_deposited_keV, center_y, center_z, unit='keV')
# plot_profile_y(energy_deposited_keV, center_x, center_z, unit='keV')
# plot_profile_z(energy_deposited_keV, center_x, center_y, unit='keV')


done
Total energy deposited: 2.7008653993854104e-11 J
Total energy deposited: 168.58801822963733 keV
